In [ ]:
import json
import pandas as pd
def process(input_str):
    return json.loads(input_str)
df_train = pd.read_csv("dataset/random_train.csv")
df_train.loc[:, 'prompt'] = df_train['prompt'].apply(process)
df_train.loc[:, 'response_a'] = df_train['response_a'].apply(process)
df_train.loc[:, 'response_b'] = df_train['response_b'].apply(process)


In [1]:
import pandas as pd

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoConfig
from transformers import EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, RobertaForMultipleChoice
import argparse
from transformers import get_polynomial_decay_schedule_with_warmup, TrainerCallback
import datasets
from datasets import Dataset

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
df_train = pd.read_csv("dataset/random_train.csv")

In [ ]:
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

In [ ]:
df_train.loc[:, 'prompt'] = df_train['prompt'].apply(process)
df_train.loc[:, 'response_a'] = df_train['response_a'].apply(process)
df_train.loc[:, 'response_b'] = df_train['response_b'].apply(process)

In [ ]:
t = df_train.loc[df_train.index % 4 != 0,].reset_index(drop = True)

In [ ]:
t.loc[20,:]

In [ ]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    return label[-1]
    
df_train = pd.read_csv('dataset/train.csv').reset_index(drop = True)
df_train['label'] = df_train.apply(lambda x: get_label(x), axis = 1)

In [ ]:
df_train.loc[df_train.index % 4 != 0,]

In [ ]:
df_train.loc[df_train.index % 20 == 0,].reset_index(drop = True).to_csv('dataset/random_valid.csv')
df_train.loc[df_train.index % 20 != 0,].reset_index(drop = True).to_csv('dataset/random_train.csv')

In [ ]:
df_train.loc[df_train.index % 20 == 0,]

# For instruction

In [2]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    if label[-1] == 0:
        return 'A'
    elif label[-1] == 1:
        return 'B'
    else:
        return 'C'
    return label[-1]

import json
import pandas as pd
def process(input_str):
    return json.loads(input_str)
train = pd.read_csv("dataset/train.csv")
train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)
train['label'] = train.apply(lambda x: get_label(x), axis = 1)

In [ ]:
# def preprocess(row):
#     row['response_a'] = row['response_a'].replace("null",'"null"')
#     row['response_b'] = row['response_b'].replace("null",'"null"')
#     if row['prompt'][-3:] == ',"]':
#         row['prompt'] = row['prompt'][:-3] + ']'
#     return row

# train= train.apply(lambda x: preprocess(x), axis = 1)

In [3]:
from tqdm import tqdm
output_as = []
output_bs = []
labels = []
ids = []
for idx, row in tqdm(train.iterrows(), total=len(train)):
    prompt = row["prompt"]
    response_a = row["response_a"]
    response_b = row["response_b"]
    label = row['label']
    id = row['id']
    assert len(prompt) == len(response_a) == len(response_b)
    for i in range(len(prompt)):
        output_as.append(f"###Model A\nPrompt: {prompt[i]}\nResponse: {response_a[i]}\n\n")
        output_bs.append(f"###Model B\nPrompt: {prompt[i]}\nResponse: {response_b[i]}\n\n")
        ids.append(id)
        labels.append(label)
        

100%|██████████| 57477/57477 [00:02<00:00, 20622.50it/s]


In [4]:
assert len(set(ids)) == len(train)
train = pd.DataFrame({'id': ids, 'instruction_a': output_as, 'instruction_b': output_bs, 'label': labels })

In [5]:
# from tqdm import tqdm
# output_as = []
# output_bs = []
# scuess_idx = []
# labels = []
# ids = []
# for idx, row in tqdm(train.iterrows(), total=len(train)):
#     try:
#         prompt = row["prompt"]
#         response_a = row["response_a"]
#         response_b = row["response_b"]
#         label = row['label']
#         id = row['id']
#         if prompt.find('","') != -1:
#             #不止一个prompt
#             #cnt = prompt.count('","')
#             prompt = row["prompt"].split('","')
#             cnt = len(prompt)
#             response_a = row["response_a"].split('","')
#             response_b = row["response_b"].split('","')
#             for sentence_idx in range(cnt):
#                 output_a = "###Model A\n"
#                 output_b = "###Model B\n"
#                 output_a += "Prompt: " + prompt[sentence_idx][2:].strip() + "\n"
#                 output_a += "Response: " + response_a[sentence_idx][2:].strip() + "\n\n"
#                 output_b += "Prompt: " + prompt[sentence_idx][2:].strip() + "\n"
#                 output_b += "Response: " +response_b[sentence_idx][2:].strip() + "\n\n"
#                 output_as.append(output_a)
#                 output_bs.append(output_b)
#                 scuess_idx.append(idx)
#                 labels.append(label)
#                 ids.append(id)
#         else:
#             output_a = "###Model A\nPrompt: " + row["prompt"][2:-2].strip() + "\nResponse: " + row["response_a"][2:-2].strip() + "\n\n"
#             output_b = "###Model B\nPrompt: " + row["prompt"][2:-2].strip() + "\nResponse: " + row["response_b"][2:-2].strip() + "\n\n"
#             output_as.append(output_a)
#             output_bs.append(output_b)
#             scuess_idx.append(idx)
#             labels.append(label)
#             ids.append(id)
        
#     except Exception as e:
#         print(f"Error: {idx}")
#         print(f"{e}")
#         output_a = "###Model A\nPrompt: " + row["prompt"][2:-2].strip() + "\nResponse: " + row["response_a"].strip() + "\n\n"
#         output_b = "###Model B\nPrompt: " + row["prompt"][2:-2].strip() + "\nResponse: " + row["response_b"].strip() + "\n\n"
#         output_as.append(output_a)
#         output_bs.append(output_b)
#         scuess_idx.append(idx)
#         labels.append(label)
#         ids.append(id)

In [6]:
idx = train.id.unique()
valid_idx = [idx[i] for i in range(len(idx)) if i % 20 == 0]

In [7]:
train.loc[train.id.isin(valid_idx),].to_csv("dataset/random_instruction_valid.csv", index = False)
train.loc[~train.id.isin(valid_idx),].to_csv("dataset/random_instruction_train.csv", index = False)

# 每个对话一段文本，做classification

In [8]:
valid = pd.read_csv("dataset/random_instruction_valid.csv")
train = pd.read_csv("dataset/random_instruction_train.csv")

In [9]:
str2num = {'A':0, "B":1, "C":2}
train['label'] = train.label.map(str2num)
valid['label'] = valid.label.map(str2num)

In [10]:
train.to_csv('dataset/random_multi_turn_cls_train.csv', index = False)
valid.to_csv('dataset/random_multi_turn_cls_valid.csv', index = False)

# 所有对话拼接一起

In [ ]:
'''
###Model A
Prompt: xxx
Response: xxx
###Model B
Prompt: xxx
Response: xxx
'''

In [11]:
valid = pd.read_csv("dataset/random_instruction_valid.csv")
train = pd.read_csv("dataset/random_instruction_train.csv")

In [12]:
%%time
# 定义一个函数来删除###Model A\n
def remove(row):
    row['instruction_a'] = row['instruction_a'].replace('###Model A\n', '')
    row['instruction_b'] = row['instruction_b'].replace('###Model B\n', '')
    return row
# 添加
def add(row):
    row['instruction_a'] = "###Model A\n" + row['instruction_a']
    row['instruction_b'] = '###Model B\n' + row['instruction_b']
    return row
    
def main(data):
    data['instruction_a'] = data['instruction_a'].apply(lambda x: x.replace('###Model A\n', ''))
    data['instruction_b'] = data['instruction_b'].apply(lambda x: x.replace('###Model B\n', ''))
    data = data.groupby('id').agg({'instruction_a': ''.join, 'instruction_b': ''.join, 'label': lambda x: list(x)[0]}).reset_index()
    data['instruction_a'] = "###Model A\n" + data['instruction_a']
    data['instruction_b'] = "###Model B\n" + data['instruction_b']
    data['prompt_response'] = data['instruction_a'] + data['instruction_b']
    return data
train = main(train)
valid = main(valid)

CPU times: user 2.19 s, sys: 224 ms, total: 2.42 s
Wall time: 2.42 s


In [13]:
train.to_csv('dataset/random_all_in_one_train.csv', index = False)
valid.to_csv('dataset/random_all_in_one_valid.csv', index = False)

all = pd.concat([train, valid]).reset_index(drop = True)
all.to_csv("dataset/full_all_in_one.csv", index = False)

In [ ]:
c = pd.read_csv('dataset/random_all_in_one_train.csv')
print(c.prompt_response.values[0])

In [14]:
# classification
str2num = {'A':0, "B":1, "C":2}
train['label'] = train.label.map(str2num)
valid['label'] = valid.label.map(str2num)

train.to_csv('dataset/random_all_in_one_cls_train.csv', index = False)
valid.to_csv('dataset/random_all_in_one_cls_valid.csv', index = False)

# 另一种prompt

In [ ]:
'''
#Prompt
xxxx
#Response
##Model A
xxxx
##Model B
xxxx

#Prompt
#Response
##Model A
xxxx
##Model B
xxxx
'''

In [4]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    if label[-1] == 0:
        return 'A'
    elif label[-1] == 1:
        return 'B'
    else:
        return 'C'
    return label[-1]

import json
import pandas as pd
def process(input_str):
    return json.loads(input_str)
train = pd.read_csv("dataset/train.csv")
train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)
train['label'] = train.apply(lambda x: get_label(x), axis = 1)

In [16]:
from tqdm import tqdm
singel_turn = []
labels = []
ids = []
for idx, row in tqdm(train.iterrows(), total=len(train)):
    prompt = row["prompt"]
    response_a = row["response_a"]
    response_b = row["response_b"]
    label = row['label']
    id = row['id']
    assert len(prompt) == len(response_a) == len(response_b)
    for i in range(len(prompt)):
        template = f"#Prompt\n{prompt[i]}\n\n#Response\n##Model A\n{response_a[i]}\n\n##Model B\n{response_b[i]}\n\n"
        singel_turn.append(template)
        ids.append(id)
        labels.append(label)
        

100%|██████████| 57477/57477 [00:02<00:00, 25199.72it/s]


In [17]:
assert len(set(ids)) == len(train)
train = pd.DataFrame({'id': ids, 'singel_turn': singel_turn,'label': labels })

In [18]:
idx = train.id.unique()
valid_idx = [idx[i] for i in range(len(idx)) if i % 20 == 0]

train.loc[train.id.isin(valid_idx),].to_csv("dataset/random_instruction_version2_valid.csv", index = False)
train.loc[~train.id.isin(valid_idx),].to_csv("dataset/random_instruction_version2_train.csv", index = False)

In [ ]:
print(train.singel_turn.values[0])

In [2]:
# 所有对话拼接在一起
valid = pd.read_csv("dataset/random_instruction_version2_valid.csv")
train = pd.read_csv("dataset/random_instruction_version2_train.csv")

In [20]:
def main(data):
    data = data.groupby('id').agg({'singel_turn': '#######\n'.join, 'label': lambda x: list(x)[0]}).reset_index()
    data = data.rename(columns = {'singel_turn':'prompt_response'})
    return data
train = main(train)
valid = main(valid)

In [21]:
train.to_csv('dataset/random_all_in_one_train_version2.csv', index = False)
valid.to_csv('dataset/random_all_in_one_valid_version2.csv', index = False)

all = pd.concat([train, valid]).reset_index(drop = True)
all.to_csv("dataset/full_all_in_one_version2.csv", index = False)

In [ ]:
print(valid.prompt_response.values[0])

In [ ]:
# 特殊例子

In [ ]:
t = pd.read_csv("dataset/random_all_in_one_train_version2.csv")
print(t.loc[t.id == 16350735,:].values)

In [13]:
from tqdm import tqdm
ids = []
for idx, row in tqdm(train.iterrows(), total=len(train)):
    prompt = row["prompt"]
    response_a = row["response_a"]
    response_b = row["response_b"]
    label = row['label']
    id = row['id']
    assert len(prompt) == len(response_a) == len(response_b)
    for i in range(len(prompt)):
        print(id)
        if 'null' in response_a[i] or 'null' in response_b[i]:
            ids.append(id)

  0%|          | 207/57477 [00:00<00:06, 8443.02it/s]

30192
30192
53567
53567
53567
65089
96401
198779
292873
313413
370945
441448
481524
481524
497862
587904
604575
604575
604575
604575
604575
604575
738614
738614
862324
863398
887722
914644
933555
1120158
1120158
1120158
1256092
1404102
1440765
1458108
1491225
1594211
1639617
1744093
1813737
1813737
1813737
1813737
1813737
1813737
1813737
1813737
1813737
1813737
1827787
1842252
1842252
1842252
1842252
1842252
2051408
2154496
2298796
2388511
2388511
2802516
2857714
2912862
2944182
3254113
3258431
3259481
3373963
3445782
3475655
3499263
3503031
3504181
3504181
3519254
3567106
3578663
3590999
3622781
3643104
3710170
3710170
3760933
3773792
3777134
3994811
3995635
3995635
3995635
3995635
3995635
4186011
4349090
4356730
4486480
4510489
4587071
4615863
4683272
4790276
4961077
4961077
4970917
4990514
5061737
5069186
5166668
5187535
5188727
5378146
5498037
5530797
5530797
5717448
5879089
5973148
6058088
6223290
6241568
6274602
6287371
6394907
6447538
6492528
6570133
6701196
6807968
6807968
6817

TypeError: argument of type 'NoneType' is not iterable

In [15]:
train.loc[208,]

id                                                         16441516
model_a                                            claude-instant-1
model_b                                                openchat-3.5
prompt            [element #1\telement #2\tcompound formed?\tche...
response_a        [element #1   element #2   compound formed?   ...
response_b        [element #1\telement #2\tcompound formed?\tche...
winner_model_a                                                    1
winner_model_b                                                    0
winner_tie                                                        0
label                                                             A
Name: 208, dtype: object

In [17]:
print(train.loc[train.id == 16350735,'response_a'].values)

[list(['I do not have any confirmed details about a World War III. Speculation about future conflicts can often be misleading or anxiety-provoking. Most experts believe maintaining open communication and cooperation between countries is the best way to prevent large-scale wars.', None])]


In [19]:
print(train.loc[train.id == 16350735,'prompt'].values)

[list(['Tell me about wwiii', 'Enough'])]


In [22]:
train.loc[train.id == 16350735,:]

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label
207,16350735,claude-instant-1,tulu-2-dpo-70b,"[Tell me about wwiii, Enough]",[I do not have any confirmed details about a W...,"[As an AI language model, I must clarify that ...",0,1,0,B


In [18]:
print(train.loc[train.id == 16350735,'response_b'].values)

[list(['As an AI language model, I must clarify that there has not been a World War III (WWIII) as of my knowledge cutoff in September 2021, and as far as I am aware, there have not been any official declarations or global conflicts that could be considered WWIII since then. Nonetheless, I can provide you with information about popular cultural references and hypothetical scenarios that involve a WWIII.\n\nIn popular culture, WWIII is often portrayed as a global conflict between major world powers, involving massive military campaigns, devastating weaponry, and significant loss of life. The causes of such a hypothetical war vary, but popular scenarios include:\n\n1. Nuclear War: A conflict between nuclear-armed nations, resulting in widespread devastation and loss of life. This is often portrayed in films, books, and video games as a potential cause of WWIII.\n\n2. Cyber Warfare: A global conflict that centers around cyber attacks and digital warfare, where nations engage in a high-sta